In [1]:

import numpy as np
import pandas as pd
import missingno as msno 
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence
from patsy import dmatrices
from sklearn.model_selection import GridSearchCV
from scipy import stats
import statsmodels.api as sm
from ISLP import confusion_table
from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
import seaborn as sns
from statsmodels.stats.stattools import durbin_watson
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

In [2]:
### 1-
train_data = pd.read_csv('E:/6th year/maia work/2. Italy - Cassino/3-Statistical Learning/Final Assignement/Data/training_data.csv' , sep=";")
test_data = pd.read_csv('E:/6th year/maia work/2. Italy - Cassino/3-Statistical Learning/Final Assignement/Data/test_data_no_target.csv', sep=";")
train_data.info

# Filter columns that exceed the threshold
missing_values_count=train_data.isnull().sum()
columns_exceeding_threshold = missing_values_count[missing_values_count > 100]


columns_to_drop = missing_values_count[missing_values_count > 100].index
train_data_cleaned = train_data.drop(columns=columns_to_drop)
test_data_cleaned = test_data.drop(columns=columns_to_drop)
train_data_cleaned.drop('Group', axis=1, inplace=True)
test_data_cleaned.drop('Group', axis=1, inplace=True)
train_data_cleaned
### 2-
def convert_decimal_separator(df):
    # Iterate through each column in the DataFrame
    for column in df.columns:
        # Replace commas with periods in the data
        df[column] = df[column].astype(str).str.replace(',', '.')
        
        # Convert the column to numeric type
        df[column] = pd.to_numeric(df[column], errors='coerce')

# Call the function to convert decimal separator in X
convert_decimal_separator(train_data_cleaned)

# Call the function to convert decimal separator in X
convert_decimal_separator(test_data_cleaned)


### 3- Null to knn
import pandas as pd
from sklearn.impute import KNNImputer

# Assuming train_data_cleaned is your DataFrame

# Display missing values by column
missing_values_train = train_data_cleaned.isnull().sum().sort_values(ascending=False)


# Initialize the KNNImputer with the 'nan_euclidean' metric
knn_imputer = KNNImputer(n_neighbors=5, metric='nan_euclidean')

# Fit and transform the data using KNN imputation
# Apply KNN imputation to the numeric columns (float64 or int64)
numeric_columns = train_data_cleaned.select_dtypes(include=['float64'])
imputed_data = knn_imputer.fit_transform(numeric_columns)

# Replace the original numeric columns in the DataFrame with the imputed data
train_data_cleaned[numeric_columns.columns] = imputed_data

# Check which columns still have missing values (this should be zero)
missing_values_after_fill = train_data_cleaned.isnull().sum().sum()





# Assuming train_data_cleaned is your DataFrame

# Display missing values by column
missing_values_test = test_data_cleaned.isnull().sum().sort_values(ascending=False)


# Initialize the KNNImputer with the 'nan_euclidean' metric
knn_imputer = KNNImputer(n_neighbors=5, metric='nan_euclidean')

# Fit and transform the data using KNN imputation
# Apply KNN imputation to the numeric columns (float64 or int64)
numeric_columns = test_data_cleaned.select_dtypes(include=['float64'])
imputed_data = knn_imputer.fit_transform(numeric_columns)

# Replace the original numeric columns in the DataFrame with the imputed data
test_data_cleaned[numeric_columns.columns] = imputed_data

# Check which columns still have missing values (this should be zero)
missing_values_after_fill_test = test_data_cleaned.isnull().sum().sum()


missing_train_values_later = train_data_cleaned.isnull().sum()

missing_test_values_later = test_data_cleaned.isnull().sum()

### 4-
target= train_data_cleaned['Perform']
classes= train_data_cleaned['Class']
train_data_cleaned.drop(['Class','Perform'], axis=1, inplace=True)


Lasso 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Store column names before splitting and scaling
original_columns = train_data_cleaned.columns

# Split the data
X_train, X_test, y_train, y_test = train_test_split(train_data_cleaned, target, test_size=0.2, random_state=42)

# Scale your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create DataFrames with scaled features
X_train_df = pd.DataFrame(X_train_scaled, columns=original_columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=original_columns)

# Now you can access column names from X_train_df, X_test_df, and y_train
print("Column names of X_train_df:")
print(X_train_df.columns)

# Instantiate LassoCV model
lasso_cv = LassoCV(cv=5)  # Use 5-fold cross-validation to find the optimal alpha

# Fit the model
lasso_cv.fit(X_train_scaled, y_train)

# Extract important features
important_features_indices = np.where(lasso_cv.coef_ != 0)[0]
important_features = train_data_cleaned.columns[important_features_indices]

print("Important features selected by Lasso:", important_features)


Column names of X_train_df:
Index(['I1', 'I2', 'I3', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I19',
       'I20', 'I25', 'I28', 'I29', 'I30', 'I31', 'I33', 'I34', 'I35', 'I36',
       'I37', 'I38', 'I39', 'I40', 'I41', 'I42', 'I43', 'I47', 'I53', 'I54',
       'I56', 'dI1', 'dI2', 'dI3', 'dI5', 'dI6', 'dI7', 'dI8', 'dI9', 'dI10',
       'dI11', 'dI19', 'dI20', 'dI25', 'dI29', 'dI30', 'dI31', 'dI33', 'dI34',
       'dI35', 'dI36', 'dI37', 'dI39', 'dI40', 'dI41', 'dI42', 'dI47', 'dI53',
       'dI54', 'dI56'],
      dtype='object')


e:\mp-anaconda3\envs\hm\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.033464017027995396, tolerance: 0.010999377067444005
  model = cd_fast.enet_coordinate_descent_gram(
e:\mp-anaconda3\envs\hm\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04109914030524919, tolerance: 0.010999377067444005
  model = cd_fast.enet_coordinate_descent_gram(
e:\mp-anaconda3\envs\hm\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0428709163082317, tolerance: 0.010999377067444005
  model = cd_fast.enet_coordinate_descent_gram(
e:\mp-anaconda3\envs\hm\Lib\site-packages\sklearn\linear_model\_coordinate_descent

Important features selected by Lasso: Index(['I6', 'I9', 'I28', 'I37', 'I41', 'I43', 'I47', 'dI6', 'dI9', 'dI25',
       'dI29', 'dI47'],
      dtype='object')


In [4]:
columns_to_select=['I6', 'I9', 'I28', 'I37', 'I41', 'I43', 'I47', 'dI6', 'dI9', 'dI25',
        'dI29', 'dI47']




# Select the columns from train_data_cleaned
new_data = train_data_cleaned[columns_to_select]
new_data_test = test_data_cleaned[columns_to_select]
# Set column names for new_data
new_data.columns = columns_to_select

# Set column names for new_data_test
new_data_test.columns = columns_to_select

In [5]:
poly = PolynomialFeatures(degree=2, include_bias=False)  # Change degree as needed
new_data = poly.fit_transform(new_data)
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import pandas as pd

# Assuming `test_cleaned_data` is the cleaned test dataset and `train_data_cleaned` was the cleaned training dataset.

# Step 1: Transform the test data using PolynomialFeatures (same instance used for training data)
X_test_interactions = poly.transform(new_data_test)

# Step 2: Convert the interaction features to a DataFrame using the column names from the training data
new_data_test = pd.DataFrame(X_test_interactions, columns=poly.get_feature_names_out(new_data_test.columns))




In [11]:
### just run


data_train, data_test, target_train, target_test = train_test_split(new_data,classes,test_size=0.2,random_state=42)

norm='std'
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Choose normalization method
norm = 'min-max'  # Or 'std' or 'none'



# Apply Min-Max normalization
if norm == 'min-max':
    scaler = MinMaxScaler()

# Apply Standard normalization
elif norm == 'std':
    scaler = StandardScaler()

# If none, keep the original data
else:
    sys.exit("ERROR: Dataset Normalization must be one of [min-max, std, none]")

# Apply normalization if required
if norm != 'none':
    X_train_scaled = scaler.fit_transform(data_train)
    X_val_scaled = scaler.transform(data_test)
    
   
else:
    X_train_scaled = data_train
    X_val_scaled = data_test


# Features indices for plotting
X_train = np.array(data_train)
X_val = np.array(data_test)



feature_idx_1 = 0
feature_idx_2 = 1



model_name='LogisticRegression'
kernel_type='sigmoid'
import sys

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter




# define KNeighborsClassifier model
if model_name == 'KNN':
    # Define the KNN model
    knn_model = KNeighborsClassifier()

    # Define hyperparameter grid for KNN
    param_grid = {'n_neighbors': range(1, 30)} # neighbors can be from 1 to 21

    # Hyperparameter tuning using cross-validation
    grid_search = GridSearchCV(knn_model, param_grid, cv=20, scoring='accuracy')
    grid_search.fit(X_train_scaled, target_train)

    # Evaluate the best combination of hyperparameters
    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)

    # Create KNN model with best parameters
    model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
    print("Model: {}\n".format(model_name)) 

# define LogisticRegression model
elif model_name == 'LogisticRegression':
    # define model
    model = LogisticRegression(max_iter=1000, random_state=42)
    print("Model: {}".format(model_name))

# define DecisionTreeClassifier model
elif model_name == 'DecisionTree':
    # define model
    model = DecisionTreeClassifier()
    print("Model: {}".format(model_name))

# define SVM model
elif model_name == 'SVM':
    if kernel_type == 'linear':
        model = SVC(kernel='linear', random_state=42)
    elif kernel_type == 'poly':
        model = SVC(kernel='poly', degree=3, random_state=42)
    elif kernel_type == 'rbf':
        model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
    elif kernel_type == 'sigmoid':
        model = SVC(kernel='sigmoid', random_state=42)
    else:
        sys.exit("ERROR: wrong kernel type [linear, poly, rbf, sigmoid]")
    print("Model: {}".format(model_name))

# define RandomForestClassifier model
elif model_name == 'RandomForest':
    # Define model with a specific random_state for reproducibility
    model = RandomForestClassifier(random_state=42)

    print("Model: {}".format(model_name))

else:
    sys.exit("ERROR: Model Creation [KNN, LogisticRegression, DecisionTree, SVM, RandomForest]")

# Train the model
# Perform self-augmentation using SMOTE
smote = SMOTE(random_state=42)
X_train_augmented, y_train_augmented = smote.fit_resample(X_train_scaled, target_train)

#model.fit(X_train_poly, target_train)#########3
model.fit(X_train_augmented, y_train_augmented)

# Perform predictions on the test set

#predictions_test = model.predict(X_test_poly)########33
predictions_test = model.predict(X_val_scaled)


def FEDCIS_accuracy(prediction, ground_truth):
    res = np.zeros(len(prediction))
    for i in range(len(prediction)):
        p = prediction[i]
        gt = ground_truth.values[i]
        if p == gt:
            res[i] = 0
        elif p == 0 and (gt == -1 or gt == 1):
            res[i] = 1
        elif (p == 1 or p == -1) and gt == 0:
            res[i] = 1
        elif p == -1 and gt == 1:
            res[i] = 2
        elif p == 1 and gt == -1:
            res[i] = 2
    return np.mean(res)



Model: LogisticRegression


In [12]:
FEDCIS_accuracy(predictions_test,target_test)

0.858125